### Node Similarity

Node similarity is an important task in graph analysis,many node similarity approacheshave been proposed in the literature. In this problemyou are asked to compute nodesimilarity using the following approaches for a Drug-Genegraph available
https://snap.stanford.edu/biodata/datasets/10002/10002-ChG-Miner.html(click onlinkChG-Miner_miner-chem-gene.tsv.gz) to downloadthe graph and identify the top 10drug-drug pairs i.e. pairs of drugs  that are similar.

*About the data*: _ChG-Miner_miner-chem-gene.tsv.gz_ is a drug-target interaction network that contains information on which genes (i.e., proteins encoded by genes) are targeted by drugs that are on the U.S. market.


In [4]:
#import statements
import pandas as pd
import networkx as nx
import re
import numpy as np;
from IPython.core.display import display
from matplotlib import pyplot as pplt

#### Q1


Create the graph

In [5]:
graph = nx.Graph()

Load the data into the graph

In [6]:
file = open('../data/ChG-Miner_miner-chem-gene.tsv')
drug_nodes = []
file.readline()
for line in file.readlines():
    vals = re.split(r'\t', line)
    drug_nodes.append(vals[0])
    graph.add_edge(vals[0], vals[1])
print(graph)
similartiy = nx.simrank_similarity(graph)
print('calculated')

Graph with 7343 nodes and 15139 edges
calculated


Print length of the keys of the dictionary

In [8]:
print(len(similartiy.keys()))

7343


Filter out the dict to only have drug nodes

In [9]:
similartiy= dict((key, val) for key, val in similartiy.items() if key.startswith('D') )

Filter out the values of the keys to have only drug nodes

In [20]:
final_similarity = dict()
for key, val in similartiy.items():
    for key1, val1 in val.items():
        if  key1.startswith('D'):
            if final_similarity.get(key) is None:
                final_similarity[key]=dict()
            final_similarity[key][key1]=val1

print(len(final_similarity.keys()))

5017


In [23]:
similarity_df = pd.DataFrame.from_dict(final_similarity)

print head of similarity_df

In [24]:
similarity_df.head()

,DB00357,DB02721,DB00773,DB07138,DB08136,DB01242,DB01238,DB00186,DB00338,DB01151,...,DB08743,DB08936,DB03185,DB04113,DB00235,DB04249,DB05679,DB06883,DB08365,DB04793
DB00357,1.000000,0.016823,0.073413,0.002779,0.002011,0.097154,0.049076,0.034902,0.120497,0.063073,...,0.018923,0.045958,0.019481,0.012626,0.037976,0.024874,0.0,0.010543,0.0,0.002368
DB02721,0.016823,1.000000,0.015868,0.002344,0.001043,0.015462,0.014756,0.021159,0.017054,0.015415,...,0.006772,0.014441,0.008653,0.391621,0.012218,0.010040,0.0,0.005326,0.0,0.001686
DB00773,0.073413,0.015868,1.000000,0.002837,0.002075,0.060316,0.039948,0.028654,0.066077,0.044258,...,0.016112,0.036828,0.016620,0.011773,0.031175,0.026332,0.0,0.009734,0.0,0.002477
DB07138,0.002779,0.002344,0.002837,1.000000,0.006213,0.002678,0.002685,0.002325,0.002901,0.002771,...,0.001880,0.002683,0.001876,0.003035,0.003968,0.006437,0.0,0.036563,0.0,0.000725
DB08136,0.002011,0.001043,0.002075,0.006213,1.000000,0.001807,0.001731,0.001514,0.002094,0.001776,...,0.001062,0.001734,0.001170,0.000890,0.001663,0.001311,0.0,0.011831,0.0,0.000501


In [31]:
def find_n_largest_nodes(dataframe, n):
    node1 = []
    node2 = []
    similarity = []
    cols = dataframe.columns
    for idx, val in enumerate(cols):
        for second_index in cols[idx+1:]:
            similarity.append(dataframe[val][second_index])
            node1.append(val)
            node2.append(second_index)

    return pd.DataFrame({'Node1': node1, 'Node2': node2, 'Similarity':similarity})
df = find_n_largest_nodes(similarity_df, 10)
display(df.nlargest(10, 'Similarity'))
display(df.nsmallest(10, 'Similarity'))

,Node1,Node2,Similarity
15055,DB07138,DB08352,0.9
15112,DB07138,DB08424,0.9
15147,DB07138,DB08095,0.9
15579,DB07138,DB08097,0.9
15639,DB07138,DB01807,0.9
16211,DB07138,DB07829,0.9
16590,DB07138,DB08423,0.9
16789,DB07138,DB01948,0.9
16810,DB07138,DB07607,0.9
17161,DB07138,DB04632,0.9


,Node1,Node2,Similarity
34,DB00357,DB04058,0.0
100,DB00357,DB07034,0.0
101,DB00357,DB04389,0.0
129,DB00357,DB07494,0.0
131,DB00357,DB07020,0.0
136,DB00357,DB05096,0.0
156,DB00357,DB07374,0.0
217,DB00357,DB02494,0.0
234,DB00357,DB04234,0.0
240,DB00357,DB07344,0.0


Jaccard Coefficient

In [48]:
jacc_coeffs =nx.jaccard_coefficient(graph)
jaccard = pd.DataFrame(jacc_coeffs, columns = ['Node1','Node2','Jaccard Coefficient'])
jaccard.head()

,Node1,Node2,Jaccard Coefficient
0,P29508\n,P10114\n,0.0
1,P29508\n,P17252\n,0.0
2,P29508\n,P49286\n,0.0
3,P29508\n,Q14627\n,0.0
4,P29508\n,DB04322,0.0


Remove gene nodes

In [49]:
first = jaccard['Node1'].astype(str)
second = jaccard['Node2'].astype(str)
jaccard = jaccard[(first.str.startswith('D') & second.str.startswith('D'))]
jaccard.nlargest(10,'Jaccard Coefficient')

,Node1,Node2,Jaccard Coefficient
31220,DB04322,DB02358,1.0
38525,DB03130,DB08179,1.0
43940,DB03130,DB03330,1.0
44994,DB02281,DB02933,1.0
49525,DB02281,DB02282,1.0
51324,DB02281,DB02158,1.0
52224,DB08010,DB08021,1.0
54530,DB08010,DB08026,1.0
54606,DB08010,DB08015,1.0
54685,DB08010,DB08023,1.0
